<a href="https://colab.research.google.com/github/deepak10pk/Image-Classification-Model/blob/master/Identify_apparels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x

In [3]:

import tensorflow as tf
tf.__version__

'2.2.0'

In [4]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Using TensorFlow backend.


In [5]:
# Reading test data from dropbox
#!wget https://www.dropbox.com/s/ywgdka2dkg1iilg/test_ScVgIM0.zip

In [6]:
#!unzip /content/test_ScVgIM0.zip

In [7]:
#Reading train data
#!wget https://www.dropbox.com/s/v96iq97ffcxgxr6/train_LbELtWX.zip

In [8]:
#!unzip /content/train_LbELtWX.zip

In [9]:
train=pd.read_csv('/content/train.csv')

In [10]:
train_image=[]
for i in tqdm(range(train.shape[0])):
  
  img =image.load_img('/content/train/'+train['id'][i].astype('str')+'.png', target_size=(28,28,1), grayscale=True)
  img = image.img_to_array( img)
  img = img/255
  train_image.append(img)
X = np.array(train_image)

  0%|          | 0/60000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 60000/60000 [00:38<00:00, 1553.75it/s]


In [11]:
X.shape

(60000, 28, 28, 1)

In [12]:
y=train['label'].values
y = to_categorical(y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [14]:
X_train.shape , X_test.shape ,y_train.shape ,y_test.shape

((48000, 28, 28, 1), (12000, 28, 28, 1), (48000, 10), (12000, 10))

In [15]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [16]:
datagen.fit(X_train)

In [22]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [33]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 17s 355us/step - loss: 0.1441 - accuracy: 0.9451 - val_loss: 0.2314 - val_accuracy: 0.9252
Epoch 2/50
48000/48000 [==============================] - 17s 349us/step - loss: 0.1404 - accuracy: 0.9465 - val_loss: 0.2404 - val_accuracy: 0.9250
Epoch 3/50
48000/48000 [==============================] - 17s 346us/step - loss: 0.1303 - accuracy: 0.9509 - val_loss: 0.2280 - val_accuracy: 0.9279
Epoch 4/50
48000/48000 [==============================] - 17s 346us/step - loss: 0.1220 - accuracy: 0.9534 - val_loss: 0.2534 - val_accuracy: 0.9246
Epoch 5/50
48000/48000 [==============================] - 17s 349us/step - loss: 0.1189 - accuracy: 0.9548 - val_loss: 0.2497 - val_accuracy: 0.9270
Epoch 6/50
48000/48000 [==============================] - 16s 341us/step - loss: 0.1137 - accuracy: 0.9578 - val_loss: 0.2508 - val_accuracy: 0.9277
Epoch 7/50
48000/48000 [==============================] 

In [34]:
test = pd.read_csv('/content/test.csv')

In [35]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/content/test/'+test['id'][i].astype('str')+'.png', target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 10000/10000 [00:03<00:00, 2851.86it/s]


In [36]:
# making predictions
prediction = model.predict(test)

In [37]:
pred=np.argmax(prediction,axis=1)

In [39]:
# creating submission file
sample = pd.read_csv('/content/sample_submission_I5njJSF.csv')
sample['label'] = pred
sample.to_csv('Deepak2_cnn.csv', header=True, index=False)